In [16]:
!pip install requests beautifulsoup4 lxml scikit-learn xgboost
!pip install selenium
!pip install webdriver-manager


In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

def scrape_naukri_jobs(role, max_jobs=10):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = f"https://www.naukri.com/{role.replace(' ', '-')}-jobs"
    driver.get(url)
    time.sleep(3)

    job_links = []
    for job in driver.find_elements(By.CSS_SELECTOR, "a.title")[:max_jobs]:
        job_links.append(job.get_attribute('href'))

    job_descriptions = []
    for link in job_links:
        try:
            driver.get(link)
            time.sleep(2)
            desc_elem = driver.find_element(By.CSS_SELECTOR, ".dang-inner")
            job_descriptions.append(desc_elem.text)
        except:
            continue

    driver.quit()

    df = pd.DataFrame(job_descriptions, columns=['Job Description'])
    return df


In [18]:
df = scrape_naukri_jobs("data analyst", max_jobs=20)
df.head()


,Job Description


In [19]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

# ✅ Skill keywords to extract
skill_keywords = [
    'python', 'sql', 'excel', 'power bi', 'machine learning', 'tensorflow',
    'r', 'tableau', 'communication', 'statistics', 'numpy', 'pandas', 'spark'
]

def extract_skills(text):
    text = str(text).lower()
    return list(set(skill for skill in skill_keywords if skill in text))

df['Extracted_Skills'] = df['Job Description'].apply(extract_skills)
df = df[df['Extracted_Skills'].map(len) > 0]

# TF-IDF + Encoding
tfidf = TfidfVectorizer(max_features=1500)
X = tfidf.fit_transform(df['Job Description'])

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Extracted_Skills'])

# Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = OneVsRestClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
model.fit(X_train, y_train)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [20]:
# 1️⃣ Fetch jobs
df_jobs = fetch_naukri_jobs("Data Analyst", pages=2)

# 2️⃣ Train model
df_with_skills = train_skill_model(df_jobs)

# 3️⃣ Predict for a new description
desc = "We are hiring a data analyst proficient in SQL, Power BI, and communication skills."
predicted = predict_skills(desc)
print("🎯 Predicted Skills:", predicted)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [21]:
print(df.columns)
print(df['Job Description'].head(10))
print(df['Job Description'].isna().sum())
print(df['Job Description'].str.len().describe())


Index(['Job Description', 'Extracted_Skills'], dtype='object')
Series([], Name: Job Description, dtype: object)
0
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Job Description, dtype: float64


In [22]:
# Remove rows with empty or short descriptions
df['Job Description'] = df['Job Description'].fillna('')
df = df[df['Job Description'].str.len() > 20]  # keep meaningful descriptions


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1500)
X = tfidf.fit_transform(df['Job Description'])


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [24]:
for i, desc in enumerate(df['Job Description'].head(10)):
    print(f"\nDescription {i+1}:\n{desc}")
